## Use Pylearn2 to Implement word/image vector fusion
- [source of both idea and code](https://github.com/mganjoo/zslearning)
- [a theano based implementation] - http://nbviewer.ipython.org/github/renruoxu/data-fusion/blob/master/deprecated/mapping%20(1).ipynb
- it is a standard 1-hidden layer MLP with customized cost function
- the data we use here is that: X (image vectors from DeCaff), Y (word vectors from word2vec)

    
##TODO

## Use Theanets to Implement Word/Image Vector fusion

In [19]:
import theanets
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [11]:
store = pd.HDFStore("../data/cifa_XY.hd5")
X = store["X/"].get_values()
y = store["Y/"].get_values()

In [146]:
train_X, other_X, train_y, other_y = train_test_split(X, y, test_size = 20000)
valid_X, test_X, valid_y, test_y = train_test_split(other_X, other_y, test_size = 10000)
print train_X.shape, valid_X.shape, test_X.shape, train_y.shape, valid_y.shape, test_y.shape

(34000, 4096) (10000, 4096) (10000, 4096) (34000, 100) (10000, 100) (10000, 100)


In [147]:
ss_X = StandardScaler()
ss_X.fit(train_X)
scaled_train_X, scaled_valid_X, scaled_test_X = ss_X.transform(train_X), ss_X.transform(valid_X), ss_X.transform(test_X)

ss_y = StandardScaler()
ss_y.fit(train_y)
scaled_train_y, scaled_valid_y, scaled_test_y = ss_y.transform(train_y), ss_y.transform(valid_y), ss_y.transform(test_y)

In [148]:
#exp.train((scaled_train_X, scaled_train_y), (scaled_valid_X, scaled_valid_y), 
#          optimize = "sgd", learning_rate = 0.005)
for train, valid in exp.itertrain(train_set = (scaled_train_X, scaled_train_y), 
                                  valid_set = (scaled_valid_X, scaled_valid_y), 
                                  optimize = "sgd", learning_rate = 0.005, validate_every = 5,
                                     hidden_l1 = 0.01, weight_l2 = 1e-4):
    print 'train loss(err)', train['loss'], "(%g)" % train["err"], 'valid loss(err)', valid['loss'], "(%g)" % valid['err']

train loss(err) 0.501571049574 (0.337281) valid loss(err) 0.507652382711 (0.34752)
train loss(err) 0.489599651034 (0.32475) valid loss(err) 0.507652382711 (0.34752)
train loss(err) 0.481531395267 (0.317602) valid loss(err) 0.507652382711 (0.34752)
train loss(err) 0.477328235284 (0.313552) valid loss(err) 0.507652382711 (0.34752)
train loss(err) 0.47394823341 (0.310065) valid loss(err) 0.507652382711 (0.34752)
train loss(err) 0.470216634923 (0.307241) valid loss(err) 0.48528451469 (0.320465)
train loss(err) 0.468957066081 (0.305537) valid loss(err) 0.48528451469 (0.320465)
train loss(err) 0.464989497482 (0.302448) valid loss(err) 0.48528451469 (0.320465)
train loss(err) 0.463798143785 (0.301249) valid loss(err) 0.48528451469 (0.320465)
train loss(err) 0.463392980533 (0.299902) valid loss(err) 0.48528451469 (0.320465)
train loss(err) 0.461434469735 (0.298907) valid loss(err) 0.486342760916 (0.321613)
train loss(err) 0.459186469714 (0.296426) valid loss(err) 0.486342760916 (0.321613)
trai

In [149]:
mean_squared_error(exp.network.predict(scaled_test_X), scaled_test_y)

0.32957093927984654

In [66]:
from gensim.models import Word2Vec
%time model = Word2Vec.load_word2vec_format("../data/word2vec.bin", binary = True)

CPU times: user 2.53 s, sys: 0 ns, total: 2.53 s
Wall time: 2.47 s


In [124]:
words = ["airplane", "automobile", "bird","cat","deer","dog","frog", "horse", "ship","truck"]
vec = []
for word in words:
    vec.append(model[word])
vec_word = zip(vec,words)

In [127]:
import numpy as np
labels_test = np.zeros((test_y.shape[0],)).astype(str)
for i in xrange(test_y.shape[0]):
    for j in xrange(len(vec_word)):
        if np.allclose(vec_word[j][0], test_y[i]):
            labels_test[i] = vec_word[j][1]
            break

In [128]:
from collections import Counter
Counter(labels_test)

Counter({'horse': 1048, 'ship': 1040, 'frog': 1009, 'bird': 1009, 'deer': 997, 'airplane': 995, 'cat': 987, 'truck': 981, 'dog': 975, 'automobile': 959})

In [133]:
from sklearn.metrics import pairwise_distances
pred_scores = exp.network.predict(scaled_test_X)
print pred_scores.shape
%time dist = pairwise_distances(pred_scores,ss_y.transform(vec), metric="euclidean")
print dist.shape
pred = np.argmin(dist,axis=1)
pred = [words[i] for i in pred]
print len(pred)

(10000, 100)
CPU times: user 2.74 ms, sys: 3.03 ms, total: 5.77 ms
Wall time: 5.78 ms
(10000, 10)
10000


In [136]:
from sklearn.metrics import accuracy_score, confusion_matrix
print accuracy_score(labels_test,pred)
print confusion_matrix(labels_test,pred)

0.7813
[[771  17  46  11  17   4   7  13  84  25]
 [ 24 840   4   2   1   1   5   0  10  72]
 [ 38   3 729  65  67  37  41  20   8   1]
 [ 12   9  61 647  35 119  63  30   7   4]
 [ 17   2  63  54 722  20  59  54   4   2]
 [  5   3  32 151  24 708  19  26   4   3]
 [ 12   2  38  56  31  16 838   4   5   7]
 [ 12   4  22  82  41  54   6 802   8  17]
 [ 54  19   8   8   5   4   2   4 895  41]
 [ 20  39   2   8   3   3  10  18  17 861]]


In [138]:
from sklearn.metrics import pairwise_distances_argmin

In [140]:
y_labels = pairwise_distances_argmin(y, vec)

In [144]:
nodog = (y_labels != 8)
nodog_X, nodog_y = X[nodog], y[nodog]
print nodog_X.shape, nodog_y.shape

(54000, 4096) (54000, 100)


In [145]:
X = nodog_X
y = nodog_y

In [150]:
store = pd.HDFStore("../data/cifa_XY.hd5")
X = store["X/"].get_values()
y = store["Y/"].get_values()
dog_X = ss_X.transform(X[~nodog])
dog_X_labels = pairwise_distances_argmin(exp.network.predict(dog_X), ss_y.transform(vec))




In [153]:
Counter(np.array(words)[dog_X_labels])

Counter({'airplane': 4165, 'truck': 905, 'automobile': 597, 'cat': 99, 'bird': 98, 'deer': 42, 'horse': 38, 'frog': 28, 'dog': 19, 'ship': 9})